# Chat with Your SFT-Trained GPT Model

This notebook allows you to interact with your fine-tuned GPT model in a conversational format. 
The model has been trained using Supervised Fine-Tuning (SFT) to respond appropriately to user messages.

## Instructions

1. **Load your trained model**: Update the model path in the first cell to point to your SFT-trained model
2. **Start chatting**: Use the chat interface to have conversations with your model
3. **Experiment**: Try different conversation styles, questions, and topics

## Features

- **Single-turn conversations**: Ask questions and get responses
- **Multi-turn conversations**: Maintain context across multiple exchanges
- **Adjustable parameters**: Control temperature and response length
- **Conversation history**: View the full conversation context

Enjoy chatting with your AI assistant! 🤖


In [ ]:
# Import necessary libraries
import torch
import sft
import gpt
from transformers import AutoTokenizer
import warnings
warnings.filterwarnings('ignore')

print("✅ Libraries imported successfully!")


In [ ]:
# Configuration - UPDATE THESE PATHS FOR YOUR MODEL

# Path to your SFT-trained model checkpoint
MODEL_PATH = "models/sft-models/sft-gpt-6000-step.pth"

# Model configuration (should match your training configuration)
MODEL_CONFIG = { 
    "vocab_size": 50262,
    "context_length": 1024,
    "emb_dim": 512,
    "n_heads": 8,
    "n_layers": 12,
    "drop_rate": 0.1,
}

# Generation parameters
MAX_NEW_TOKENS = 150
TEMPERATURE = 0.7

print(f"📁 Model path: {MODEL_PATH}")
print(f"⚙️  Model config: {MODEL_CONFIG}")
print(f"🎛️  Max tokens: {MAX_NEW_TOKENS}, Temperature: {TEMPERATURE}")


In [ ]:
# Load tokenizer and model
print("🔄 Loading tokenizer...")
tokenizer = gpt.setup_tokenizer()
print(f"✅ Tokenizer loaded! Vocab size: {tokenizer.vocab_size}")

print("🔄 Loading model...")
model = sft.load_pretrained_model(MODEL_PATH, MODEL_CONFIG)

# Move to GPU if available
device = 'cuda:5' if torch.cuda.is_available() else 'cpu'
model = model.to(device)
print(f"✅ Model loaded and moved to {device}!")


## Single-Turn Chat Interface

Use the function below to have individual conversations with your model. Each call is independent.


In [ ]:
def chat_once(user_message, max_tokens=MAX_NEW_TOKENS, temperature=TEMPERATURE):
    """
    Have a single conversation turn with the model.

    Args:
        user_message: Your message to the model
        max_tokens: Maximum number of tokens to generate
        temperature: Sampling temperature (higher = more creative)

    Returns:
        Model's response
    """
    print(f"👤 You: {user_message}")
    print("🤖 Assistant: ", end="")

    response = sft.generate_chat_response(
        model=model,
        tokenizer=tokenizer,
        user_message=user_message,
        max_new_tokens=max_tokens,
        temperature=temperature
    )

    print(response)
    print("\n" + "="*50 + "\n")
    return response

# Test the chat function
print("🧪 Testing chat function...")
test_response = chat_once("Hello! How are you today?")
print("✅ Chat function working!")


## Multi-Turn Chat Interface

Use the class below to maintain conversation context across multiple turns.


In [ ]:
class ChatSession:
    """
    A chat session that maintains conversation history.
    """

    def __init__(self, model, tokenizer, max_tokens=MAX_NEW_TOKENS, temperature=TEMPERATURE):
        self.model = model
        self.tokenizer = tokenizer
        self.max_tokens = max_tokens
        self.temperature = temperature
        self.conversation_history = []

    def add_message(self, role, content):
        """Add a message to the conversation history."""
        self.conversation_history.append({"role": role, "content": content})

    def chat(self, user_message):
        """
        Send a message and get a response, maintaining conversation context.

        Args:
            user_message: Your message to the model

        Returns:
            Model's response
        """
        # Add user message to history
        self.add_message("user", user_message)

        print(f"👤 You: {user_message}")
        print("🤖 Assistant: ", end="")

        # Generate response using conversation history
        response = sft.generate_multi_turn_response(
            model=self.model,
            tokenizer=self.tokenizer,
            conversation_history=self.conversation_history[:-1],  # Exclude the current user message
            max_new_tokens=self.max_tokens,
            temperature=self.temperature
        )

        # Add assistant response to history
        self.add_message("assistant", response)

        print(response)
        print("\n" + "="*50 + "\n")
        return response

    def clear_history(self):
        """Clear the conversation history."""
        self.conversation_history = []
        print("🗑️  Conversation history cleared!")

    def show_history(self):
        """Display the full conversation history."""
        print("📜 Conversation History:")
        print("="*50)
        for i, msg in enumerate(self.conversation_history):
            role_emoji = "👤" if msg["role"] == "user" else "🤖"
            print(f"{role_emoji} {msg['role'].title()}: {msg['content']}")
            if i < len(self.conversation_history) - 1:
                print("\n")
        print("="*50)

    def set_parameters(self, max_tokens=None, temperature=None):
        """Update generation parameters."""
        if max_tokens is not None:
            self.max_tokens = max_tokens
        if temperature is not None:
            self.temperature = temperature
        print(f"⚙️  Updated parameters: max_tokens={self.max_tokens}, temperature={self.temperature}")

# Create a chat session
chat_session = ChatSession(model, tokenizer)
print("✅ Chat session created!")


## Interactive Chat Examples

Try these example conversations to test your model!


In [ ]:
# Example 1: Single-turn conversation
print("🎯 Example 1: Single-turn conversation")
chat_once("What is machine learning?")


In [ ]:
# Example 2: Multi-turn conversation
print("🎯 Example 2: Multi-turn conversation")

# Start a new conversation
chat_session.clear_history()

# First turn
chat_session.chat("Hi! I'm learning about AI. Can you help me?")

# Second turn (maintains context)
chat_session.chat("What's the difference between supervised and unsupervised learning?")

# Third turn (continues the conversation)
chat_session.chat("Can you give me an example of each?")

# Show the full conversation history
chat_session.show_history()


In [ ]:
# Example 3: Creative conversation
print("🎯 Example 3: Creative conversation")

# Adjust parameters for more creative responses
chat_session.set_parameters(max_tokens=200, temperature=0.9)

# Start a creative conversation
chat_session.clear_history()
chat_session.chat("Write a short story about a robot learning to paint.")
chat_session.chat("What happens next in the story?")

# Show the creative conversation
chat_session.show_history()


## Your Turn to Chat!

Now it's your turn to experiment with your model. Try different types of conversations:

- **Questions**: Ask about various topics
- **Creative tasks**: Ask for stories, poems, or creative writing
- **Problem solving**: Present problems and ask for solutions
- **Role-playing**: Have the model take on different personas
- **Multi-turn**: Build complex conversations with context

Use the functions below to chat with your model!


In [ ]:
# Single-turn chat - try your own messages!
chat_once("Your message here!")


In [ ]:
# Multi-turn chat - start a conversation!
chat_session.clear_history()
chat_session.chat("Your first message here!")


In [ ]:
# Continue the conversation
chat_session.chat("Your follow-up message here!")


In [ ]:
# Adjust parameters for different response styles
print("🎛️  Adjusting parameters...")

# More conservative responses
chat_session.set_parameters(max_tokens=100, temperature=0.3)
chat_session.chat("Explain quantum computing simply.")

# More creative responses
chat_session.set_parameters(max_tokens=200, temperature=1.0)
chat_session.chat("Write a haiku about programming.")


In [ ]:
# View your conversation history
chat_session.show_history()


## Tips for Better Conversations

### Parameter Tuning
- **Temperature (0.1 - 1.0)**: Lower = more focused, Higher = more creative
- **Max tokens (50 - 500)**: Controls response length

### Conversation Strategies
- **Be specific**: Clear questions get better answers
- **Provide context**: Multi-turn conversations work better with context
- **Experiment**: Try different conversation styles and topics
- **Iterate**: Adjust parameters based on the responses you get

### Common Issues
- **Short responses**: Increase `max_tokens`
- **Repetitive responses**: Adjust `temperature`
- **Off-topic responses**: Provide more context or be more specific

## Model Information

Your model was trained using:
- **Architecture**: GPT with RoPE positional encoding
- **Training**: Supervised Fine-Tuning on conversational data
- **Special tokens**: `<|user|>`, `<|assistant|>`, `<|end|>`
- **Context length**: 1024 tokens

Enjoy exploring your AI assistant! 🚀
